# Playground

## GNN DEMO
This code receives multiple timeseries, transforms them into graphs, and then applies a GNN to them.
The graph embeddings are then used for downstream tasks.

In [1]:
%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
import pytorch_lightning as pl

In [2]:
# DATA
class MyDataModule(pl.LightningDataModule):
    def prepare_data(self):
        X_input = torch.randn(50, 128*60, 58)
        X_output = torch.randn(1, 128*60, 58).repeat(50, 1, 1)
        y_class = torch.randint(low=0, high=3, size=(50,))
        y_text = torch.randn(50, 1024)

        self.train_dataset = torch.utils.data.TensorDataset(X_input, X_output, y_class, y_text)
        self.val_dataset = torch.utils.data.TensorDataset(X_input, X_output, y_class, y_text)
        
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=32)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=32)

In [18]:
# MODEL
class MyModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=58, hidden_size=2, num_layers=1, batch_first=True)
        self.fc = nn.Linear(2, 3)
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        h, y = self.rnn(x)
        y = self.fc(y)
        y = y.permute(1, 0, 2).squeeze(1)
        # y = torch.argmax(y, dim=2).to(torch.float)
        return y
    
    def training_step(self, batch, batch_idx):
        X_input, X_output, y_class, y_text = batch
        y_hat = self(X_input)
        loss = self.loss(y_hat, y_class)
        self.log('train/loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        X_input, X_output, y_class, y_text = batch
        y_hat = self(X_input)
        loss = self.loss(y_hat, y_class)
        self.log('val/loss', loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [19]:
# TRAINING
model = MyModel()
datamodule = MyDataModule()
trainer = pl.Trainer(max_epochs=50, accelerator='cpu', logger=True, log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type             | Params
------------------------------------------
0 | rnn  | RNN              | 124   
1 | fc   | Linear           | 9     
2 | loss | CrossEntropyLoss | 0     
------------------------------------------
133       Trainable params
0         Non-trainable params
133       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]